In [1]:
%load_ext autoreload
%autoreload 2
import json
import sqlite3
import datetime
from app import app, db
from app.models import User, Review, Business, Category, Feature
from app.review_tools import extract_nouns, stars_prior
db.create_all()

import nltk

import pandas as pd
from textblob import TextBlob

/Users/mayen/anaconda/envs/python35/lib/python3.5/site-packages/flask_sqlalchemy/__init__.py:800: UserWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.
  warnings.warn('SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.')


Creating DB


In [2]:
reviews = pd.read_sql_table('reviews',app.config['SQLALCHEMY_DATABASE_URI'])
business = pd.read_sql_table('business',app.config['SQLALCHEMY_DATABASE_URI'])
category = pd.read_sql_table('category',app.config['SQLALCHEMY_DATABASE_URI'])

In [3]:
pizza_reviews = reviews.merge(category[category['category']=='Pizza'],
                              how='inner',
                              on='business_id',
                             copy=False)
pizza_reviews.drop(['id_x','id_y'],axis=1,inplace=True)

In [4]:
review = pizza_reviews.iloc[123]
extract_nouns(review)

{'negative': {},
 'positive': {'food': 1,
  'hoarse': 1,
  'i': 1,
  'notch': 1,
  'penza': 1,
  'place': 1}}

In [5]:
def noun_count(revs):
    """revs is reviews for a particular business"""
    word_count = {'positive':{}, 'negative':{}}
    for i,rev in revs.iterrows():
        nouns = extract_nouns(rev)
        for sentiment, counts in nouns.items():
            for word,count in counts.items():
                word_count[sentiment][word] = word_count[sentiment].get(word,0)+count
    return word_count
    
def populate(reviews):
    businesses = reviews['business_id'].unique()
    pos = 'noun'
    for bid in businesses:
        print(bid)
        revs = reviews[reviews['business_id']==bid]
        word_count = noun_count(revs)
        for sentiment, counts in word_count.items():
            positive = int(sentiment == 'positive')
            for phrase,count in counts.items():
                feature = Feature(bid, phrase, pos, positive, count)
                db.session.add(feature)
                db.session.commit()
    print('Success!')
    return 1

In [6]:
from multiprocessing import Pool

In [9]:
def populate_by_bid(bid):
    revs = pizza_reviews[pizza_reviews['business_id']==bid]
    word_count = noun_count(revs)
    pos = 'noun'
    for sentiment, counts in word_count.items():
        positive = int(sentiment == 'positive')
        for phrase,count in counts.items():
            feature = Feature(bid, phrase, pos, positive, count)
            db.session.add(feature)
            db.session.commit()
    return 1

In [10]:
bids = pizza_reviews['business_id'].unique()
p = Pool(6)
p.map(populate_by_bid,bids)

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [20]:
word_count

{'negative': {'': 2,
  'accent': 1,
  'adjustment': 1,
  'amount': 2,
  'ample': 1,
  'answer': 1,
  'anyhow': 1,
  'anything': 2,
  'appeal': 1,
  'apple': 2,
  'apron': 1,
  'area': 5,
  'ask': 1,
  'atmosphere': 1,
  'attitude': 1,
  'b': 1,
  'bah': 3,
  'bathroom': 1,
  'bear': 1,
  'beer': 2,
  'before': 1,
  'bell': 1,
  'berg': 1,
  'bet': 17,
  'bit': 3,
  'bland': 1,
  'booth': 1,
  'boothstable': 1,
  'born': 1,
  'box': 7,
  'boy': 1,
  'boyfriend': 1,
  'bread': 2,
  'breadstick': 1,
  'broadway': 1,
  'bunch': 1,
  'busines': 2,
  'cafeterium': 1,
  'car': 3,
  'carry': 1,
  'category': 1,
  'cent': 1,
  'chair': 1,
  'chance': 2,
  'change': 2,
  'charm': 1,
  'check': 1,
  'cheese': 54,
  'chicago': 1,
  'chicken': 1,
  'childhood': 1,
  'chink': 1,
  'choice': 1,
  'city': 1,
  'close': 1,
  'cold': 11,
  'coleslaw': 1,
  'complaint': 1,
  'conclusion': 1,
  'corner': 1,
  'craving': 1,
  'crust': 11,
  'crutch': 1,
  'cup': 1,
  'customer': 1,
  'cut': 9,
  'cycle': 1